In [1]:

from collections import namedtuple
from PIL import Image, ImageDraw, ImageFont
from google.cloud import vision
from enum import Enum
%matplotlib inline
import matplotlib.pyplot as plt
import os
import json
from google.protobuf.json_format import MessageToJson
from google.protobuf import json_format

import sys
sys.path.append("..")
from models.bounding_box import FeatureType, Point, BoundingBox
from utils.cv_preprocess import draw_boxes, group_bounding_boxes, merge_box_groups

from utils.file_utils import prepare_image_local, prepare_image_web, load_counter, save_counter, save_json

from utils.nlp_preprocess import load_spacy_models, is_english, is_chinese, split_dish_info, is_word_relevant

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../menu-ocr-390814-9e6224d09feb.json'


f:\Anaconda\envs\Translation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
filein = '../downloads/Ma-La-Menu_07-2023.pdf'
base_name = os.path.basename(filein)
file_name_without_extension = os.path.splitext(base_name)[0]


raw_ocr_directory = '../output/raw_ocr/'
raw_ocr_filename = file_name_without_extension + "_raw_annotation.json"
raw_ocr_path = os.path.join(raw_ocr_directory, raw_ocr_filename)

preprocessed_ocr_directory = '../output/prep_ocr/'
preprocessed_ocr_filename = file_name_without_extension + "_prep_ocr.json"
preprocessed_ocr_path = os.path.join(preprocessed_ocr_directory, preprocessed_ocr_filename)

In [3]:
raw_ocr_path

'../output/raw_ocr/Ma-La-Menu_07-2023_raw_annotation.json'

In [4]:
image = prepare_image_local(filein)
# Same page
# image = prepare_image_local('./images/bluefrog蓝蛙(189弄购物中心店)_menu_photos_8.jpg')

In [5]:
client = vision.ImageAnnotatorClient()

In [6]:
response = client.document_text_detection(
    image=image, 
    image_context={"language_hints": ["zh", "en"]}
)

# Load existing counter
current_count = load_counter()

# Increment counter
current_count += 1

# Save updated counter
save_counter(current_count)

In [7]:
document_json = MessageToJson(response.full_text_annotation._pb)

if not os.path.exists(raw_ocr_directory):
    os.makedirs(raw_ocr_directory)

with open(raw_ocr_path, 'w', encoding='utf-8') as json_file:
    json_file.write(document_json)